**Load and Configure Model**
- โหลดโมเดลที่เทรนไว้แล้ว
- ปรับ `FC Layers` ให้ตรงกับโมเดล
- โหลดน้ำหนักโมเดลที่ฝึกไว้
- ตั้งค่าอุปกรณ์ให้พร้อมใช้งาน `eval form`

In [ ]:
import cv2
import torch
from torchvision import transforms, models
from PIL import Image
import numpy as np

# Load Model
model = models.resnet50(pretrained=False)  
model.fc = torch.nn.Linear(model.fc.in_features, 8) 
model_save_path = "plastic_model 4.pth"  
model.load_state_dict(torch.load(model_save_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

# Configure
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) 
model.eval()

**Model with OpenCV**
- **Preprocess Image**:
  - ปรับขนาดและทำการแปลงภาพให้อยู่ในรูปแบบที่โมเดลรองรับ
  
- **Predict Function**:
  - คำนวณความน่าจะเป็นของพลาสติกแต่ละประเภท
  - รวมประเภทพลาสติกบางชนิดเป็น `Non Recyclable`
- **การแสดงผลจากกล้อง**:
  - ใช้กล้องเพื่อแสดงผล `prediction` แบบเรียลไทม์บนหน้าจอ

In [2]:
# Preprocess image
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocess_image(frame):
    image = transform(frame)
    image = image.unsqueeze(0)
    return image

# Labels to group as "Non Recyclable"
non_recyclable_labels = {
    "UHT Box",
    "Single Use Plastic",
    "Multi layer Plastic",
    "bottle waste bow bag"
}

# Function for predictions
def predict(model, frame, labels):
    with torch.no_grad():
        image = preprocess_image(frame).to(device)
        outputs = model(image)
        probabilities = torch.sigmoid(outputs).squeeze(0)  # Calculate probabilities
        predictions = (probabilities > 0.5).float()  # threshold
        
        # Group results
        results = []
        for label, prob in zip(labels, probabilities):
            if predictions[labels.index(label)] == 1:
                if label in non_recyclable_labels:
                    results.append(("Non Recyclable", prob.item()))
                else:
                    results.append((label, prob.item()))
        return results

# Plastic Type
labels = [
    "PET Bottle",
    "HDPE Plastic",
    "Single layer Plastic",
    "Squeeze Tube",
    "UHT Box",
    "Single Use Plastic",
    "Multi layer Plastic",
    "bottle waste bow bag"
]

# Open-Cam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open the camera")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot read frame from the camera")
        break

    # Convert BGR to RGB (cause model expects RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Predictions
    predictions = predict(model, rgb_frame, labels)

    # Display results on screen
    output_text = "Predictions: " + ", ".join([f"{label} ({prob:.2f})" for label, prob in predictions])
    cv2.putText(frame, output_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow('Plastic Waste Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()